In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import time

import numpy as np
import tensorflow as tf


C:\Users\bsubra1\.conda\envs\uberocrenv\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\bsubra1\.conda\envs\uberocrenv\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\bsubra1\.conda\envs\uberocrenv\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\bsubra1\.conda\envs\uberocrenv\lib\site-packages\ten

In [2]:
def load_graph(model_file):
  graph = tf.Graph()
  graph_def = tf.GraphDef()

  with open(model_file, "rb") as f:
    graph_def.ParseFromString(f.read())
  with graph.as_default():
    tf.import_graph_def(graph_def)

  return graph

In [3]:
def load_labels(label_file):
  label = []
  proto_as_ascii_lines = tf.gfile.GFile(label_file).readlines()
  for l in proto_as_ascii_lines:
    label.append(l.rstrip())
  return label


In [6]:
#graph=load_graph('C:/biju/Experiments/aimemory/tensorflow-for-poets-2-LandM/tf_files/training_summaries/InceptionV3/retrained_graph.pb')


graph=load_graph('C:\\ocr\\customcode\\imageclassification_trainer\\tensorflow-for-poets-2\\tf_files\\training_summaries\\InceptionV3\\retrained_graph.pb')

In [8]:
def read_tensor_from_image_file(file_name, input_height=299, input_width=299,
				input_mean=0, input_std=255):
  input_name = "file_reader"
  output_name = "normalized"
  file_reader = tf.read_file(file_name, input_name)
  if file_name.endswith(".png"):
    image_reader = tf.image.decode_png(file_reader, channels = 3,
                                       name='png_reader')
  elif file_name.endswith(".gif"):
    image_reader = tf.squeeze(tf.image.decode_gif(file_reader,
                                                  name='gif_reader'))
  elif file_name.endswith(".bmp"):
    image_reader = tf.image.decode_bmp(file_reader, name='bmp_reader')
  else:
    image_reader = tf.image.decode_jpeg(file_reader, channels = 3,
                                        name='jpeg_reader')
  float_caster = tf.cast(image_reader, tf.float32)
  dims_expander = tf.expand_dims(float_caster, 0);
  resized = tf.image.resize_bilinear(dims_expander, [input_height, input_width])
  normalized = tf.divide(tf.subtract(resized, [input_mean]), [input_std])
  sess = tf.Session()
  result = sess.run(normalized)

  return result

In [6]:
res=read_tensor_from_image_file('C:\\ocr\\BOL\\ImageClassifier\\Signature\\3NB format 1-example-381.jpg')

In [32]:
import cv2
#frame=cv2.imread('C:\\ocr\\BOL\\ImageClassifier\\Signature\\3NB format 1-example-381.jpg')
#frame=cv2.imread('C:/ocr/BOL/BOLSamples/NB format 1-example-4.jpg')
frame=cv2.imread('C:/ocr/BOL/BOLSamples/NB format 1-example-11.jpg')

In [33]:
frame.shape

(5500, 4250, 3)

In [34]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import time
import cv2
import numpy as np
import tensorflow as tf

input_height=299
input_width=299
input_mean=0
input_std=255

def load_graph(model_file):
    graph = tf.Graph()
    graph_def = tf.GraphDef()

    with open(model_file, "rb") as f:
        graph_def.ParseFromString(f.read())
    with graph.as_default():
        tf.import_graph_def(graph_def)

    return graph

def load_labels(label_file):
    label = []
    proto_as_ascii_lines = tf.gfile.GFile(label_file).readlines()
    for l in proto_as_ascii_lines:
        label.append(l.rstrip())
    return label

#graph=load_graph('C:/biju/Experiments/aimemory/tensorflow-for-poets-2-LandM/tf_files/training_summaries/InceptionV3/retrained_graph.pb')
graph=load_graph('C:\\ocr\\customcode\\imageclassification_trainer\\tensorflow-for-poets-2\\tf_files\\training_summaries\\InceptionV3\\retrained_graph.pb')

img2= cv2.resize(frame,dsize=(299,299), interpolation = cv2.INTER_CUBIC)
#Numpy array
np_image_data = np.asarray(img2)
#maybe insert float convertion here - see edit remark!
np_final = np.expand_dims(np_image_data,axis=0)
#float_caster 

#= tf.cast(np_final, tf.float32)
#dims_expander = tf.expand_dims(float_caster, 0);
resized = tf.image.resize_bilinear(np_final, [input_height, input_width])
normalized = tf.divide(tf.subtract(resized, [input_mean]), [input_std])
sess = tf.Session()
t = sess.run(normalized)

input_layer = "Mul"
output_layer = "final_result"

input_name = "import/" + input_layer
output_name = "import/" + output_layer
input_operation = graph.get_operation_by_name(input_name);
output_operation = graph.get_operation_by_name(output_name);

with tf.Session(graph=graph) as sess:
    start = time.time()
    results = sess.run(output_operation.outputs[0],
                      {input_operation.outputs[0]: t})
    end=time.time()
results = np.squeeze(results)

top_k = results.argsort()[-5:][::-1]
#labels = load_labels('C:/biju/Experiments/aimemory/tensorflow-for-poets-2-LandM/tf_files/training_summaries/InceptionV3/retrained_labels.txt')

labels = load_labels('C:\\ocr\\customcode\\imageclassification_trainer\\tensorflow-for-poets-2\\tf_files\\training_summaries\\InceptionV3\\retrained_labels.txt')
print('\nEvaluation time (1-image): {:.3f}s\n'.format(end-start))
template = "{} (score={:0.5f})"
for i in top_k:
    print(template.format(labels[i], results[i]))


Evaluation time (1-image): 1.284s

signature (score=0.77543)
nosignature (score=0.22457)


In [21]:
def read_tensor_from_frame(frame, input_height=299, input_width=299,
				input_mean=0, input_std=255):
  input_name = "file_reader"
  output_name = "normalized"
  file_reader = tf.read_file(file_name, input_name)
  if file_name.endswith(".png"):
    image_reader = tf.image.decode_png(file_reader, channels = 3,
                                       name='png_reader')
  elif file_name.endswith(".gif"):
    image_reader = tf.squeeze(tf.image.decode_gif(file_reader,
                                                  name='gif_reader'))
  elif file_name.endswith(".bmp"):
    image_reader = tf.image.decode_bmp(file_reader, name='bmp_reader')
  else:
    image_reader = tf.image.decode_jpeg(file_reader, channels = 3,
                                        name='jpeg_reader')
  float_caster = tf.cast(image_reader, tf.float32)
  dims_expander = tf.expand_dims(float_caster, 0);
  resized = tf.image.resize_bilinear(dims_expander, [input_height, input_width])
  normalized = tf.divide(tf.subtract(resized, [input_mean]), [input_std])
  sess = tf.Session()
  result = sess.run(normalized)

  return result